## softmax regression의  cost function 구현하기


1. F.softmax() + torch.log() = F.log_softmax()

In [2]:
import torch
import torch.nn.functional as F

In [10]:
z = torch.rand(3, 5, requires_grad=True)
print(z)
y = torch.randint(5, (3,)).long()
# torch.log(F.softmax(z, dim=1))
F.log_softmax(z, dim=1)

tensor([[0.5359, 0.8606, 0.8324, 0.3124, 0.6616],
        [0.8242, 0.1709, 0.2753, 0.0664, 0.2045],
        [0.8560, 0.4198, 0.0416, 0.8258, 0.3143]], requires_grad=True)


tensor([[-1.7338, -1.4091, -1.4373, -1.9573, -1.6082],
        [-1.1329, -1.7862, -1.6818, -1.8907, -1.7527],
        [-1.2924, -1.7286, -2.1068, -1.3225, -1.8340]],
       grad_fn=<LogSoftmaxBackward>)

2. F.log_softmax() + F.nll_loss() = F.cross_entropy()

In [9]:
# (y_one_hot * -torch.log(F.softmax(z, dim=1))).sum(dim=1).mean() # low-level로 구현한 비용함수
F.nll_loss(F.log_softmax(z, dim=1), y) # Negative Log Likelihood. 원핫벡터를 널을 필요 없이 바로 실제값을 인자로 사용

tensor(1.6914, grad_fn=<NllLossBackward>)

In [7]:
F.cross_entropy(z, y)           # torch.nn.CrossEntropyLoss() 과 같음

tensor(1.6197, grad_fn=<NllLossBackward>)

## softmax regression 구현하기

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [10]:
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_train = [2, 2, 2, 1, 1, 1, 0, 0]
x_train = torch.FloatTensor(x_train)    # 총 8개의 샘플이 존재하며, 각 샘플은 4개의 특성을 가짐
y_train = torch.LongTensor(y_train)     # 각 샘플에 대한 레이블이며, 여기서는 0, 1, 2  총 3개의 클래스가 존재함

In [12]:
y_one_hot = torch.zeros(8, 3)         # 클래스의 개수는 3개이므로 y_train에 원-핫 인코딩한 결과는 8 × 3의 개수를 가져야 함
y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
print(y_one_hot.shape)

torch.Size([8, 3])


In [ ]:
W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

optimizer = optim.SGD([W, b], lr=0.1)

<low - level>

In [ ]:
nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # 가설
    hypothesis = F.softmax(x_train.matmul(W) + b, dim=1) 

    # 비용 함수
    cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

<High level>

<high - level>

In [ ]:
nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # Cost 계산
    z = x_train.matmul(W) + b
    cost = F.cross_entropy(z, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

<nn.Module로 구현하기>

In [13]:
# 모델을 선언 및 초기화. 4개의 특성을 가지고 3개의 클래스로 분류. input_dim=4, output_dim=3.
model = nn.Linear(4, 3)

In [14]:
optimizer = optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = F.cross_entropy(prediction, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 20번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.480385
Epoch  100/1000 Cost: 0.714275
Epoch  200/1000 Cost: 0.624039
Epoch  300/1000 Cost: 0.563412
Epoch  400/1000 Cost: 0.511285
Epoch  500/1000 Cost: 0.462559
Epoch  600/1000 Cost: 0.415360
Epoch  700/1000 Cost: 0.368847
Epoch  800/1000 Cost: 0.322791
Epoch  900/1000 Cost: 0.278408
Epoch 1000/1000 Cost: 0.245999


<Class로 구현하기>
위의 방법과 차이라고 할 수 있는 것은 model = nn.Linear(4, 3) 부분을 클래스 정의할 때 self.linear = nn.Linear(4, 3)부분으로 넣어줘서 
model = SoftmaxClassifierModel()이렇게 한것 밖에 없다. 

In [15]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(4, 3) # Output이 3!

    def forward(self, x):
        return self.linear(x)

In [16]:
model = SoftmaxClassifierModel()

In [17]:
optimizer = optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = F.cross_entropy(prediction, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 20번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 3.143817
Epoch  100/1000 Cost: 0.712404
Epoch  200/1000 Cost: 0.629844
Epoch  300/1000 Cost: 0.572103
Epoch  400/1000 Cost: 0.521461
Epoch  500/1000 Cost: 0.473762
Epoch  600/1000 Cost: 0.427411
Epoch  700/1000 Cost: 0.381635
Epoch  800/1000 Cost: 0.336118
Epoch  900/1000 Cost: 0.291341
Epoch 1000/1000 Cost: 0.252489
